In [4]:

import pandas as pd
import numpy as np
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))
from database.connect_db import connect


In [ ]:
# Connect database
query = "SELECT * FROM bronze.raw_data"
conn, cur = connect()
df = pd.read_sql_query(query, conn)

Database connected successfully!


C:\Users\Gray\AppData\Local\Temp\ipykernel_46056\3984953757.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


#### Khám phá dữ liệu (EDA)
---

In [8]:
df.columns

Index(['id', 'title', 'address', 'area', 'floors', 'furniture', 'bedrooms',
       'bathrooms', 'price', 'price_m2', 'posted_date', 'link'],
      dtype='object')

**Notes:**
- ``floors:`` Số tầng
- ``furniture:`` nội thất


In [9]:
# Preview data
df.head()

,id,title,address,area,floors,furniture,bedrooms,bathrooms,price,price_m2,posted_date,link
0,1,"DUY NHẤT - NHÀ 3 TẦNG KIÊN CỐ, SẴN DÒNG TIỀN 3...","Phường 13, Tân Bình, Hồ Chí Minh",101 m²,None,Full nội thất,None,None,"17,5 tỷ","~173,27 triệu/m²",17/11/2025,https://batdongsan.com.vn/ban-nha-rieng-phuong...
1,2,"Chính chủ bán nhà riêng, hoàn công 2023, SHR, ...","Đường Tân Thới Nhất 17, Phường Tân Thới Nhất, ...",84 m²,3 tầng,Đầy đủ,4 phòng,4 phòng,"12,7 tỷ","~151,19 triệu/m²",17/11/2025,https://batdongsan.com.vn/ban-nha-rieng-duong-...
2,3,"Chú VINH CẦN BÁN CĂN NHÀ 3,68Tỷ Ở CHU VĂN AN, ...","Chu Văn An, 12, Bình Thạnh, Hồ Chí Minh 3,68Tỷ","51,2 m²",2 tầng,None,4 phòng,4 phòng,"3,68 tỷ","~71,88 triệu/m²",16/11/2025,https://batdongsan.com.vn/ban-nha-rieng-duong-...
3,4,Bán nhà mới xây Gò Vấp 6 tầng có thang máy gar...,"Đường Quang Trung, Phường 10, Gò Vấp, Hồ Chí Minh",85 m²,6 tầng,Cao cấp,5 phòng,8 phòng,"16,5 tỷ","~194,12 triệu/m²",20/10/2025,https://batdongsan.com.vn/ban-nha-rieng-duong-...
4,5,Nhà hẻm xe hơi - 2 lầu full nội thất - Huỳnh T...,"Đường Huỳnh Tấn Phát, Phường Phú Thuận, Quận 7...",60 m²,3 tầng,Full nội thất,4 phòng,4 phòng,"7,49 tỷ","~124,83 triệu/m²",16/11/2025,https://batdongsan.com.vn/ban-nha-rieng-duong-...


Cột id chỉ là cột đánh dấu số thứ tự trong database nên không cần thiết vì vậy ta sẽ drop nó đi và set index lại giống id.

In [19]:
df.set_index("id",inplace=True)

In [20]:
df.head()

,title,address,area,floors,furniture,bedrooms,bathrooms,price,price_m2,posted_date,link
id,,,,,,,,,,,
1,"DUY NHẤT - NHÀ 3 TẦNG KIÊN CỐ, SẴN DÒNG TIỀN 3...","Phường 13, Tân Bình, Hồ Chí Minh",101 m²,None,Full nội thất,None,None,"17,5 tỷ","~173,27 triệu/m²",17/11/2025,https://batdongsan.com.vn/ban-nha-rieng-phuong...
2,"Chính chủ bán nhà riêng, hoàn công 2023, SHR, ...","Đường Tân Thới Nhất 17, Phường Tân Thới Nhất, ...",84 m²,3 tầng,Đầy đủ,4 phòng,4 phòng,"12,7 tỷ","~151,19 triệu/m²",17/11/2025,https://batdongsan.com.vn/ban-nha-rieng-duong-...
3,"Chú VINH CẦN BÁN CĂN NHÀ 3,68Tỷ Ở CHU VĂN AN, ...","Chu Văn An, 12, Bình Thạnh, Hồ Chí Minh 3,68Tỷ","51,2 m²",2 tầng,None,4 phòng,4 phòng,"3,68 tỷ","~71,88 triệu/m²",16/11/2025,https://batdongsan.com.vn/ban-nha-rieng-duong-...
4,Bán nhà mới xây Gò Vấp 6 tầng có thang máy gar...,"Đường Quang Trung, Phường 10, Gò Vấp, Hồ Chí Minh",85 m²,6 tầng,Cao cấp,5 phòng,8 phòng,"16,5 tỷ","~194,12 triệu/m²",20/10/2025,https://batdongsan.com.vn/ban-nha-rieng-duong-...
5,Nhà hẻm xe hơi - 2 lầu full nội thất - Huỳnh T...,"Đường Huỳnh Tấn Phát, Phường Phú Thuận, Quận 7...",60 m²,3 tầng,Full nội thất,4 phòng,4 phòng,"7,49 tỷ","~124,83 triệu/m²",16/11/2025,https://batdongsan.com.vn/ban-nha-rieng-duong-...


Sau khi set id thành index thì id sẽ không còn là cột trong dataframe nữa

#### Feature Classification (Phân loại đặc trưng)
---
``Mục đích:`` Chọn đúng biểu đồ để trực quan hóa và chọn đúng kỹ thuật tiền xử lý dữ liệu.

1/ Đặc trưng định tính(Categorical Features)

Các đặc trưng định tính(hay còn gọi là biến ohân loại) là những biến mà giá trị của chúng là các nhãn(labels) hoặc danh mục(categories).

``Các loại phổ biến:``
- ``Nominal(Danh nghĩa):`` Các danh mục không có thứ tự hoặc xếp hạng tự nhiên.
    Ví dụ: Màu sắc(Xanh, đỏ, vàng), quốc gia(Việt Nam, Lào)
- ``Ordinal(Thứ tự):`` Các danh mục có thứ tự hoặc xếp hạng tự nhiên.
    Ví dụ: Mức độ hài lòng(Rất tệ, tệ), cấp bậc

2/ Đặc trưng định lượng(Numerical Features)

Các đặc trưng định lượng(hay còn gọi là biến số) là những biến mà giá trị của chúng là các số và có ý nghĩa toán học.

``Các loại phổ biến:``
- ``Discrete(rời rạc):`` Các giá trị là số nguyên, kết quả của việc đếm.
    Ví dụ: Số lượng phòng ngủ, số lần nhấp chuột, số lượng học sinh
- ``Continuous(Liên tục):`` Các giá trị có thể là bất kỳ số nào trong một phạm vi, kết quả của việc đo lường.
    Ví dụ: Diện tích, giá, nhiệt độ, độ cao.
- ``Timeseries(chuỗi thời gian):`` Là một dạng liên tục đặc biệt, được sắp xếp theo trình tự thời gian.
    Ví dụ: Gía cổ phiếu hàng ngày, nhiệt độ trung bình, ngày đăng tin.

``TẠI SAO VIỆC PHÂN LOẠI LẠI QUAN TRỌNG?``

1/ Chọn biểu đồ trực quan hóa (Visualization):
- ``Định lượng:`` Thường dùng Histogram, Box Plot, Scatter Plot.
- ``Định tính:`` Thường dùng Bar Chart(Biểu đồ thanh), Pie Chart.

2/ Xử lý giá trị thiếu(Missing values):
- ``Định lượng:`` Thường thay thế bằng giá trị Trung bình(Mean), hoặc trung vị (Median).
- ``Định tính:`` Thường thay thế bằng Mode(Gía trị xuất hiện nhiều nhất) hoặc một danh mục mới("Unknow")

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2814 entries, 1 to 2814
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        2791 non-null   object
 1   address      2791 non-null   object
 2   area         2791 non-null   object
 3   floors       2380 non-null   object
 4   furniture    1619 non-null   object
 5   bedrooms     2349 non-null   object
 6   bathrooms    2238 non-null   object
 7   price        2791 non-null   object
 8   price_m2     2628 non-null   object
 9   posted_date  2791 non-null   object
 10  link         2814 non-null   object
dtypes: object(11)
memory usage: 263.8+ KB
